# 诗歌生成

# 数据处理

In [12]:
import numpy as np
import tensorflow as tf
import collections
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, datasets

start_token = 'bos'
end_token = 'eos'

def process_dataset(fileName):
    examples = []
    with open(fileName, 'r') as fd:
        for line in fd:
            outs = line.strip().split(':')
            content = ''.join(outs[1:])
            ins = [start_token] + list(content) + [end_token] 
            if len(ins) > 200:
                continue
            examples.append(ins)
            
    counter = collections.Counter()
    for e in examples:
        for w in e:
            counter[w]+=1
    
    sorted_counter = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*sorted_counter)
    words = ('PAD', 'UNK') + words[:len(words)]
    word2id = dict(zip(words, range(len(words))))
    id2word = {word2id[k]:k for k in word2id}
    
    indexed_examples = [[word2id[w] for w in poem]
                        for poem in examples]
    seqlen = [len(e) for e in indexed_examples]
    
    instances = list(zip(indexed_examples, seqlen))
    
    return instances, word2id, id2word

def poem_dataset():
    instances, word2id, id2word = process_dataset('./poems.txt')
    ds = tf.data.Dataset.from_generator(lambda: [ins for ins in instances], 
                                            (tf.int64, tf.int64), 
                                            (tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.shuffle(buffer_size=10240)
    ds = ds.padded_batch(100, padded_shapes=(tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.map(lambda x, seqlen: (x[:, :-1], x[:, 1:], seqlen-1))
    return ds, word2id, id2word

# 模型代码， 完成建模代码

In [13]:
class myRNNModel(keras.Model):
    def __init__(self, w2id):
        super(myRNNModel, self).__init__()
        self.v_sz = len(w2id)
        self.embed_layer = tf.keras.layers.Embedding(self.v_sz, 64, 
                                                    batch_input_shape=[None, None])
        
        self.rnncell = tf.keras.layers.SimpleRNNCell(128)
        self.rnn_layer = tf.keras.layers.RNN(self.rnncell, return_sequences=True)
        self.dense = tf.keras.layers.Dense(self.v_sz)
        
    @tf.function
    def call(self, inp_ids):
        '''
        前向传播逻辑
        Args:
            inp_ids: shape(batch_size, seq_len), dtype=tf.int32
        Returns:
            logits: shape(batch_size, seq_len, vocab_size), dtype=tf.float32
        '''
        # 嵌入层：将输入 ID 映射到高维空间
        embed = self.embed_layer(inp_ids)  # shape: (batch_size, seq_len, emb_dim)

        # RNN 层：处理序列数据
        rnn_output = self.rnn_layer(embed)  # shape: (batch_size, seq_len, hidden_dim)

        # 全连接层：输出每个时间步的分类概率
        logits = self.dense(rnn_output)  # shape: (batch_size, seq_len, vocab_size)

        return logits
    
    @tf.function
    def get_next_token(self, x, state):
        '''
        shape(x) = [b_sz,] 
        '''
    
        inp_emb = self.embed_layer(x) #shape(b_sz, emb_sz)
        h, state = self.rnncell.call(inp_emb, state) # shape(b_sz, h_sz)
        logits = self.dense(h) # shape(b_sz, v_sz)
        out = tf.argmax(logits, axis=-1)
        return out, state

## 一个计算sequence loss的辅助函数，只需了解用途。

In [14]:
def mkMask(input_tensor, maxLen):
    shape_of_input = tf.shape(input_tensor)
    shape_of_output = tf.concat(axis=0, values=[shape_of_input, [maxLen]])

    oneDtensor = tf.reshape(input_tensor, shape=(-1,))
    flat_mask = tf.sequence_mask(oneDtensor, maxlen=maxLen)
    return tf.reshape(flat_mask, shape_of_output)


def reduce_avg(reduce_target, lengths, dim):
    """
    Args:
        reduce_target : shape(d_0, d_1,..,d_dim, .., d_k)
        lengths : shape(d0, .., d_(dim-1))
        dim : which dimension to average, should be a python number
    """
    shape_of_lengths = lengths.get_shape()
    shape_of_target = reduce_target.get_shape()
    if len(shape_of_lengths) != dim:
        raise ValueError(('Second input tensor should be rank %d, ' +
                         'while it got rank %d') % (dim, len(shape_of_lengths)))
    if len(shape_of_target) < dim+1 :
        raise ValueError(('First input tensor should be at least rank %d, ' +
                         'while it got rank %d') % (dim+1, len(shape_of_target)))

    rank_diff = len(shape_of_target) - len(shape_of_lengths) - 1
    mxlen = tf.shape(reduce_target)[dim]
    mask = mkMask(lengths, mxlen)
    if rank_diff!=0:
        len_shape = tf.concat(axis=0, values=[tf.shape(lengths), [1]*rank_diff])
        mask_shape = tf.concat(axis=0, values=[tf.shape(mask), [1]*rank_diff])
    else:
        len_shape = tf.shape(lengths)
        mask_shape = tf.shape(mask)
    lengths_reshape = tf.reshape(lengths, shape=len_shape)
    mask = tf.reshape(mask, shape=mask_shape)

    mask_target = reduce_target * tf.cast(mask, dtype=reduce_target.dtype)

    red_sum = tf.reduce_sum(mask_target, axis=[dim], keepdims=False)
    red_avg = red_sum / (tf.cast(lengths_reshape, dtype=tf.float32) + 1e-30)
    return red_avg

# 定义loss函数，定义训练函数

In [15]:
@tf.function
def compute_loss(logits, labels, seqlen):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)
    losses = reduce_avg(losses, seqlen, dim=1)
    return tf.reduce_mean(losses)

@tf.function
def train_one_step(model, optimizer, x, y, seqlen):
    with tf.GradientTape() as tape:
        # 前向传播
        logits = model(x)  # shape: (batch_size, seq_len, vocab_size)
        # 计算损失
        loss = compute_loss(logits, y, seqlen)

    # 计算梯度并更新参数
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

def train(epoch, model, optimizer, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y, seqlen) in enumerate(ds):
        loss = train_one_step(model, optimizer, x, y, seqlen)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy())

    return loss

# 训练优化过程

In [16]:
optimizer = optimizers.Adam(0.0005)
train_ds, word2id, id2word = poem_dataset()
model = myRNNModel(word2id)

for epoch in range(10):
    loss = train(epoch, model, optimizer, train_ds)

epoch 0 : loss 8.819752
epoch 1 : loss 6.5525346
epoch 2 : loss 6.119895
epoch 3 : loss 5.8418603
epoch 4 : loss 5.696637
epoch 5 : loss 5.601245
epoch 6 : loss 5.4207983
epoch 7 : loss 5.3971877
epoch 8 : loss 5.4013667
epoch 9 : loss 5.267058


# 生成过程

In [17]:
import random

def generate_poem(begin_word, model, w2id, id2w, poem_type=random.choice([5,7])):
    state = model.rnncell.get_initial_state(batch_size=1, dtype=tf.float32)
    current_word = tf.constant([w2id[begin_word]], dtype=tf.int32)
    poem_lines = []
    current_line = [begin_word]
    
    # 忽略的符号列表（模型可能生成的无效字符）
    banned_tokens = ['，', '。', 'eos', 'UNK']
    
    for _ in range(4 * poem_type * 4):  # 放宽生成次数限制
        current_word, state = model.get_next_token(current_word, state)
        word = id2w.get(current_word.numpy()[0], 'UNK')
        
        # 关键逻辑：过滤无效符号
        if word in banned_tokens:
            continue  # 直接跳过，不加入诗句
        
        current_line.append(word)
        
        # 每句长度达标后添加标点
        if len(current_line) == poem_type:
            punctuation = '，' if len(poem_lines) < 3 else '。'
            poem_lines.append(''.join(current_line) + punctuation)
            current_line = []
        
        # 生成四句后强制终止
        if len(poem_lines) >= 4:
            break
    
    # 补全最后一句（如果未完成）
    if len(poem_lines) < 4 and current_line:
        current_line = current_line[:poem_type]  # 截断至规定长度
        punctuation = '。' if len(poem_lines) >= 3 else '，'
        poem_lines.append(''.join(current_line) + punctuation)
    
    # 确保最终为四句
    return ''.join(poem_lines[:4])

begin_words = ["日", "红", "山", "夜", "湖", "海", "月"]

for word in begin_words:
    poem = generate_poem(word, model, word2id, id2word, poem_type=random.choice([5,7]))  # 生成七言诗
    print(f"【{word}】开头生成的诗：\n{poem}\n")


【日】开头生成的诗：
日暮花声人，不见一人不，知心不可知，子不可知不。

【红】开头生成的诗：
红氲畔畔帆递怅，蓉猿递怅忆湘江，水上山看一枝来，不得人不得不知。

【山】开头生成的诗：
山上水风雨落花，声子无人事无人，不可知心无限处，不见此中心客无。

【夜】开头生成的诗：
夜暮花声落，风吹落水寒，子无人事何，人不可知心。

【湖】开头生成的诗：
湖上春风吹落日，如明人不得无人，事不得人间不得，人道不知无限处。

【海】开头生成的诗：
海阳年子无人一，人无人来不得不，得人间生不得不，得无人生不得不。

【月】开头生成的诗：
月秋月不知不知，何处是何人来不，得人相见不得人，间不得人得不知。

